In [2]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

In [3]:
data = pd.read_csv("sentences.csv")

In [4]:
data = data.dropna()

# Display the first few rows to understand the structure
data.head()

,darija,eng,darija_ar
0,"homa mkhbbyin chi haja, ana mti99en!","They're hiding something, I'm sure!","هوما مخبّيين شي حاجة, أنا متيقّن!"
1,bayna homa tay7awlo ib9aw mbrrdin.,It's obvious they're trying to keep their cool.,باينا هوما تايحاولو إبقاو مبرّدين.
2,loTilat mabaynach fihom mori7in bzzaf.,the hotels don't seem very comfortable.,لوطيلات مابايناش فيهوم موريحين بزّاف.
3,ghaliban ghayjrriw 3lih mn lkhdma!,he is probably about to be laid off by head of...,غاليبان غايجرّيو عليه من لخدما!
4,Tab3an rah mkta2eb!,of course he's depressive!,طابعان راه مكتاءب!


In [5]:
data['english'] = data['eng']
data

,darija,eng,darija_ar,english
0,"homa mkhbbyin chi haja, ana mti99en!","They're hiding something, I'm sure!","هوما مخبّيين شي حاجة, أنا متيقّن!","They're hiding something, I'm sure!"
1,bayna homa tay7awlo ib9aw mbrrdin.,It's obvious they're trying to keep their cool.,باينا هوما تايحاولو إبقاو مبرّدين.,It's obvious they're trying to keep their cool.
2,loTilat mabaynach fihom mori7in bzzaf.,the hotels don't seem very comfortable.,لوطيلات مابايناش فيهوم موريحين بزّاف.,the hotels don't seem very comfortable.
3,ghaliban ghayjrriw 3lih mn lkhdma!,he is probably about to be laid off by head of...,غاليبان غايجرّيو عليه من لخدما!,he is probably about to be laid off by head of...
4,Tab3an rah mkta2eb!,of course he's depressive!,طابعان راه مكتاءب!,of course he's depressive!
...,...,...,...,...
12742,ra deja chet imta 5aykoun lfilm,i've already checked the showtimes,را دجا شت إمتا خايكون لفيلم,i've already checked the showtimes
12743,nhar l7ed 5aydar flil m3etel,there's a late night screening on saturday,نهار لحد خايدار فليل معتل,there's a late night screening on saturday
12744,ana kangoul nmchiw nt3echaw 9bel,i'm thinking we could grab some dinner before ...,أنا كانڭول نمشيو نتعشاو قبل,i'm thinking we could grab some dinner before ...
12745,blan nadi,that's a great plan,بلان نادي,that's a great plan


In [6]:
from sklearn.model_selection import train_test_split

# Assuming the CSV has two columns: 'English' and 'Darija'
english_sentences = data['english'].tolist()
darija_sentences = data['darija'].tolist()

# Split the data into training, validation, and test sets
train_eng, temp_eng, train_dar, temp_dar = train_test_split(english_sentences, darija_sentences, test_size=0.2)
val_eng, test_eng, val_dar, test_dar = train_test_split(temp_eng, temp_dar, test_size=0.5)

In [7]:
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.layers import Embedding, LSTM, Dense, Input, Bidirectional
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam

# Tokenize the sentences
tokenizer_eng = Tokenizer()
tokenizer_eng.fit_on_texts(train_eng)
train_eng_seq = tokenizer_eng.texts_to_sequences(train_eng)
val_eng_seq = tokenizer_eng.texts_to_sequences(val_eng)

tokenizer_dar = Tokenizer()
tokenizer_dar.fit_on_texts(train_dar)
train_dar_seq = tokenizer_dar.texts_to_sequences(train_dar)
val_dar_seq = tokenizer_dar.texts_to_sequences(val_dar)

In [8]:
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences


# Convert the data to lowercase and remove any special characters
data['english'] = data['english'].str.lower().str.replace('[^\w\s]', '', regex=True)
data['darija'] = data['darija'].str.lower().str.replace('[^\w\s]', '', regex=True)

# Add special tokens to the sentences
data['english'] = data['english'].apply(lambda x: 'starttoken ' + x + ' endtoken')
data['darija'] = data['darija'].apply(lambda x: 'starttoken ' + x + ' endtoken')

# Initialize tokenizers and add special tokens explicitly
english_tokenizer = Tokenizer(filters='')
darija_tokenizer = Tokenizer(filters='')

# Manually add the special tokens
english_tokenizer.fit_on_texts(['starttoken', 'endtoken'])
darija_tokenizer.fit_on_texts(['starttoken', 'endtoken'])

# Fit the tokenizers on the actual text data
english_tokenizer.fit_on_texts(data['english'])
darija_tokenizer.fit_on_texts(data['darija'])

# Convert text to sequences
english_sequences = english_tokenizer.texts_to_sequences(data['english'])
darija_sequences = darija_tokenizer.texts_to_sequences(data['darija'])



# Pad the sequences
max_eng = [len(seq) for seq in english_sequences]
max_english_length = 0
for i in max_eng:
    if i>max_english_length:
        max_english_length=i

max_dar = [len(seq) for seq in darija_sequences]
max_darija_length = 0
for i in max_dar:
    if i>max_darija_length:
        max_darija_length=i
        
english_sequences = pad_sequences(english_sequences, maxlen=max_english_length, padding='post')
darija_sequences = pad_sequences(darija_sequences, maxlen=max_darija_length, padding='post')

<>:7: SyntaxWarning: invalid escape sequence '\w'
<>:8: SyntaxWarning: invalid escape sequence '\w'
<>:7: SyntaxWarning: invalid escape sequence '\w'
<>:8: SyntaxWarning: invalid escape sequence '\w'
C:\Users\Administrator\AppData\Local\Temp\ipykernel_7980\1562620273.py:7: SyntaxWarning: invalid escape sequence '\w'
  data['english'] = data['english'].str.lower().str.replace('[^\w\s]', '', regex=True)
C:\Users\Administrator\AppData\Local\Temp\ipykernel_7980\1562620273.py:8: SyntaxWarning: invalid escape sequence '\w'
  data['darija'] = data['darija'].str.lower().str.replace('[^\w\s]', '', regex=True)


In [9]:
from sklearn.model_selection import train_test_split

# Split the data
X_train, X_val, y_train, y_val = train_test_split(english_sequences, darija_sequences, test_size=0.2, random_state=42)


In [10]:
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, LSTM, Dense, Embedding

# Define the encoder
encoder_inputs = Input(shape=(max_english_length,))
encoder_embedding = Embedding(len(english_tokenizer.word_index) + 1, 256, mask_zero=True)(encoder_inputs)
encoder_lstm, state_h, state_c = LSTM(256, return_state=True)(encoder_embedding)
encoder_states = [state_h, state_c]

# Define the decoder
decoder_inputs = Input(shape=(max_darija_length,))
decoder_embedding = Embedding(len(darija_tokenizer.word_index) + 1, 256, mask_zero=True)(decoder_inputs)
decoder_lstm = LSTM(256, return_sequences=True, return_state=True)
decoder_outputs, _, _ = decoder_lstm(decoder_embedding, initial_state=encoder_states)
decoder_dense = Dense(len(darija_tokenizer.word_index) + 1, activation='softmax')
decoder_outputs = decoder_dense(decoder_outputs)

# Define the model
model = Model([encoder_inputs, decoder_inputs], decoder_outputs)
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy')

# Print the model summary
model.summary()


Model: "functional_1"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer         │ (None, 40)        │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ input_layer_1       │ (None, 31)        │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ embedding           │ (None, 40, 256)   │  1,417,472 │ input_layer[0][0] │
│ (Embedding)         │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ not_equal           │ (None, 40)        │          0 │ input_layer[0][0] │
│ (NotEqual)          │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ embedding_1         │ (None, 31, 256)   │  3,857,408 │ input_layer_1[0]… │
│ (Embedding)         │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ lstm (LSTM)         │ [(None, 256),     │    525,312 │ embedding[0][0],  │
│                     │ (None, 256),      │            │ not_equal[0][0]   │
│                     │ (None, 256)]      │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ lstm_1 (LSTM)       │ [(None, 31, 256), │    525,312 │ embedding_1[0][0… │
│                     │ (None, 256),      │            │ lstm[0][1],       │
│                     │ (None, 256)]      │            │ lstm[0][2]        │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense (Dense)       │ (None, 31, 15068) │  3,872,476 │ lstm_1[0][0]      │
└─────────────────────┴───────────────────┴────────────┴───────────────────┘

 Total params: 10,197,980 (38.90 MB)

 Trainable params: 10,197,980 (38.90 MB)

 Non-trainable params: 0 (0.00 B)

In [11]:
# Prepare target data by shifting the sequences
y_train_shifted = np.zeros_like(y_train)
y_train_shifted[:, :-1] = y_train[:, 1:]

y_val_shifted = np.zeros_like(y_val)
y_val_shifted[:, :-1] = y_val[:, 1:]

# Train the model
model.fit([X_train, y_train], y_train_shifted, validation_data=([X_val, y_val], y_val_shifted), epochs=10, batch_size=64)


Epoch 1/10
160/160 ━━━━━━━━━━━━━━━━━━━━ 209s 1s/step - loss: 7.3753 - val_loss: 5.9137
Epoch 2/10
160/160 ━━━━━━━━━━━━━━━━━━━━ 203s 1s/step - loss: 5.5381 - val_loss: 5.9135
Epoch 3/10
160/160 ━━━━━━━━━━━━━━━━━━━━ 203s 1s/step - loss: 5.2961 - val_loss: 5.8921
Epoch 4/10
160/160 ━━━━━━━━━━━━━━━━━━━━ 175s 1s/step - loss: 5.0660 - val_loss: 5.9029
Epoch 5/10
160/160 ━━━━━━━━━━━━━━━━━━━━ 212s 1s/step - loss: 4.8533 - val_loss: 5.9255
Epoch 6/10
160/160 ━━━━━━━━━━━━━━━━━━━━ 180s 1s/step - loss: 4.6695 - val_loss: 5.9392
Epoch 7/10
160/160 ━━━━━━━━━━━━━━━━━━━━ 168s 1s/step - loss: 4.4372 - val_loss: 5.9650
Epoch 8/10
160/160 ━━━━━━━━━━━━━━━━━━━━ 176s 1s/step - loss: 4.2116 - val_loss: 6.0061
Epoch 9/10
160/160 ━━━━━━━━━━━━━━━━━━━━ 165s 1s/step - loss: 3.9837 - val_loss: 6.0548
Epoch 10/10
160/160 ━━━━━━━━━━━━━━━━━━━━ 198s 1s/step - loss: 3.7737 - val_loss: 6.1035


In [12]:
# Evaluate the model on the validation set
loss = model.evaluate([X_val, y_val], y_val_shifted)
print(f'Validation Loss: {loss}')

80/80 ━━━━━━━━━━━━━━━━━━━━ 15s 180ms/step - loss: 6.1806
Validation Loss: 6.101609230041504


In [21]:
def translate(sentence):
    # Preprocess the sentence
    sentence = 'starttoken ' + sentence.lower().replace('[^\w\s]', '') + ' endtoken'
    sequence = english_tokenizer.texts_to_sequences([sentence])
    sequence = pad_sequences(sequence, maxlen=max_english_length, padding='post')

    # Encode the sentence
    states = encoder_model.predict(sequence)
    
    # Initialize the target sequence with the start token
    target_seq = np.zeros((1, 1))
    target_seq[0, 0] = darija_tokenizer.word_index['starttoken']
    
    translated_sentence = []
    
    for _ in range(max_darija_length):
        # Predict the next word
        output_tokens, h, c = decoder_model.predict([target_seq] + states)
        
        # Get the highest probability word index
        word_id = np.argmax(output_tokens[0, -1, :])
        word = darija_tokenizer.index_word.get(word_id, '')
        
        if word == 'endtoken':
            break
        
        translated_sentence.append(word)
        
        # Update the target sequence and states
        target_seq = np.zeros((1, 1))
        target_seq[0, 0] = word_id
        states = [h, c]

    return ' '.join(translated_sentence)

# Define the encoder model for prediction
encoder_model = Model(encoder_inputs, encoder_states)

# Define the decoder model for prediction
decoder_state_input_h = Input(shape=(256,))
decoder_state_input_c = Input(shape=(256,))
decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]

decoder_outputs, state_h, state_c = decoder_lstm(
    decoder_embedding, initial_state=decoder_states_inputs)
decoder_states = [state_h, state_c]
decoder_outputs = decoder_dense(decoder_outputs)

decoder_model = Model(
    [decoder_inputs] + decoder_states_inputs,
    [decoder_outputs] + decoder_states)

# Translate a sample sentence
print(translate('how i can help you?').replace('starttoken ',''))


<>:3: SyntaxWarning: invalid escape sequence '\w'
<>:3: SyntaxWarning: invalid escape sequence '\w'
C:\Users\Administrator\AppData\Local\Temp\ipykernel_7980\1957419781.py:3: SyntaxWarning: invalid escape sequence '\w'
  sentence = 'starttoken ' + sentence.lower().replace('[^\w\s]', '') + ' endtoken'


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 260ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 222ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 38ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step
wach kayn chi 7aja


In [22]:
# Save the entire model
model.save('translation_model.h5')

In [23]:
import pickle

# Save English tokenizer
with open('english_tokenizer.pkl', 'wb') as f:
    pickle.dump(english_tokenizer, f)

# Save Darija tokenizer
with open('darija_tokenizer.pkl', 'wb') as f:
    pickle.dump(darija_tokenizer, f)

# Save maximum sequence lengths
with open('max_seq_lengths.pkl', 'wb') as f:
    pickle.dump({'max_english_length': max_english_length, 'max_darija_length': max_darija_length}, f)
